# 6장 : 텍스트와 시퀀스를 위한 딥러닝 

#### 구성
* 텍스트 데이터 다루기
* 순환 신경망 이해하기
* 순환 신경망의 고급 사용법
* 컨브넷을 이용한 시퀀스 처리



#### 이 장에서 다룰 핵심 내용 
* 텍스트 데이터를 유용한 형태로 전처리 하는 방법
* 순환신경망을 사용하는 방법
* 1D 컨브넷을 사용한 데이터 처리

---



*텍스트 (단어의 시퀀스 또는 문자의 시퀀스), 시계열 또는 일반적인 시퀀스 데이터를 처리할 수 있는 딥러닝 모델, **순환 신경망(recurrent neural network)**, **1D 컨브넷 (1D Convnet)** 을 살펴보겠습니다.*
* 문서 분류나 시계열 분류
* 시계열 비교
* 시퀀스-투-시퀀스 학습 (ex. 영어 문장을 프랑스 어로 변환하기)
* 감성 분석 (트윗이나 영화 리뷰가 긍정적인지 부정적인지 분류하기)
* 시계열 예측 (최근 날씨 데이터가 주어졌을 때 향후 날씨를 예측하기)

## 6.1 텍스트 데이터 다루기

텍스트는 가장 흔한 시퀀스 형태의 데이터이다. 보통 단어 수준으로 작업하는 경우가 많다. 딥러닝 모델은 수치형 텐서만 다룰 수 있으므로, 텍스트를 수치형 텐서로 변환하는 **텍스트 벡터화** 과정을 거쳐야 합니다. 이에는 여러가지 방식이 있습니다. 
* 텍스트를 단어로 나누고 각 단어를 하나의 벡터로 변환합니다.
* 텍스트를 문자로 나누고 각 문자를 하나의 벡터로 변환합니다.
* 텍스트에서 단어나 문자의 n-gram을 추출하여 각 n-gram을 하나의 벡터로 변환합니다. n-gram은 연속된 단어나 문자의 그룹으로 텍스트에서 단어나 문자를 하나씩 이동하면서 추출합니다.

토큰(token) : 텍스트를 나누는 단위 (단어, 문자, n-gram), 그리고 텍스트를 토큰으로 나누는 작업을 토큰화(tokenization)이라고 합니다.

토큰과 벡터를 연결하는 방법은 여러가지가 있습니다.
* 원-핫 인코딩 (One-hot encoding)
* 토큰 임베딩 (Token embedding)


### n-gram & BoW
BoW가 **순서가 없는** 토큰화 방법이기 때문에 딥러닝 모델보다 얕은 학습방법의 언어 처리모델에 사용되는 경향이 있습니다. 시퀀스가 아니라 집합으로 간주되고 문장의 일반적인 구조가 사라지기 때문입니다. n-gram을 추출하는 것은 일종의 특성 공학 입니다. 딥러닝은 유연하지 못하고 불안정한 이런 방식을 계층적인 특성학습으로 대체합니다. (n-gram은 로지스틱 회기나 랜덤포레스트같은 얕은 학습방법의 텍스트 처리 모델을 사용할 때 강력하고 아주 유용한 특성 공학 입니다.)


`The cat sat on the mat`


* bag of 2-gram

`{"The", "The cat", "cat", "cat sat", "sat", "sat on", "on", "on the", "the", "the mat", "mat"}`


* bag of 3-gram

`{"The", "The cat", "cat", "cat sat", "The cat sat", "sat", "sat on", "on", "cat sat on", "on the", "the", "sat on the", "the mat", "mat", "on the mat"}`


### 단어와 문자의 원-핫 인코딩

In [5]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
            
max_length = 10

results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1
        
print(results)

[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [8]:
import string

characters = string.printable
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1

print(results)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


케라스에는 원본 텍스트 데이터를 단어 또는 문자 수준의 원-핫 인코딩으로 변환해 주는 유틸리티가 있습니다. 특수 문자를 제거하거나 빈도가 높은 N개의 단어만 선택하는 등 여러가지 중요한 기능들이 있기 때문에 이 유틸리티를 사용하는 것이 좋습니다.

In [11]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
print(len(word_index))

9


원-핫 인코딩의 변종 중 하나는 **원-핫 해싱(One-hot Hashing)** 기법입니다. 이 방식은 어휘 사전에 있는 고유한 토큰의 수가 너무 커서 모두 다루기 어려울 때 사용합니다. 각 단어에 명시적으로 인덱스를 할당하고 이 인덱스를 딕셔너리에 저장하는 대신에 단어를 해싱하여 고정된 크기의 벡터로 변환합니다. 일반적으로 간단한 해싱 함수를 사용합니다. 이 방식의 주요 장점은 명시적인 단어 인덱스가 필요없기 때문에 메모리를 절약하고 온라인 방식으로 데이터를 인코딩할 수 있습니다. (**전체 데이터를 확인하지 않고 토큰을 생성할 수 있습니다.**) 

한 가지 단점은 해시 충돌(Hash Collision) 입니다. 2개의 단어가 같은 해시를 만들면 이를 바라보는 머신 러닝 모델은 단어 사이의 차이를 인식 하지 못합니다. 해싱 공간의 차원이 해싱될 고유 토큰의 전체 개수보다 훨씬 크면 해싱 출돌의 가능성이 줄어듭니다.

In [14]:
dimensionality = 1000
max_length     = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1
print(results)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


### 단어 임베딩 사용하기

